### Data Collection

- Data is to be found on website.
- So we will download and store it localy first.

In [1]:
import os

In [2]:
%pwd

'd:\\ML\\xyz_employee_welness\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\ML\\xyz_employee_welness'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: list
    data_dir: list

In [7]:
from empWellness.constants import *
from empWellness.utils.common import read_yaml, create_directries
from empWellness.logging import logger
import requests

In [8]:
class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directries([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            data_dir=config.data_dir,
        )

        return data_ingestion_config

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def ingest(self):
        try:
            logger.info("Ingestion started!")

            create_directries([self.config.root_dir])

            file_name = ["train.csv", "test.csv", "submission.csv"]

            for i in range(len(self.config.source_URL)):
                file_path = os.path.join(self.config.root_dir, self.config.data_dir[i], file_name[i])
                # print(file_path)

                if not os.path.exists(file_path):
                    file_dir = os.path.join(
                        self.config.root_dir, self.config.data_dir[i]
                    )
                    create_directries([file_dir])
                    logger.info(f"created dirctory: {file_dir[i]} for {file_name[i]}")

                    response = requests.get(self.config.source_URL[i])
                    raw_data = response.content.decode(
                        encoding="utf-8", errors="ignore"
                    )
                    logger.info(f"File: {file_name[i]} downloaded!")

                    with open(file_path, "w") as f:
                        f.write(raw_data)
                        logger.info(f"File: {file_name[i]} saved to {file_dir[i]}")
                        
                else:
                    logger.info(f"File: {file_name} already exists!")

        except Exception as e:
            raise e

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.ingest()
except Exception as e:
    raise e